In [ ]:
cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [2]:
from timeit import default_timer
class Timer(object):
    def __init__(self, msg='Starting.....', timer=default_timer, factor=1,
                 fmt="------- elapsed {:.4f}s --------"):
        self.timer = timer
        self.factor = factor
        self.fmt = fmt
        self.end = None
        self.msg = msg

    def __call__(self):
        return self.timer()

    def __enter__(self):
        print(self.msg)
        self.start = self()
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.end = self()
        print(str(self))

    def __repr__(self):
        return self.fmt.format(self.elapsed)

    def elapsed(self):
        if self.end is None:
            return (self() - self.start) * self.factor
        else:
            return (self.end - self.start) * self.factor

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from attacks import cw


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
img_size = 28
img_chan = 1
n_classes = 10
batch_size = 32


print('\nLoading MNIST')

dataset = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = dataset.load_data()
X_train = np.reshape(X_train, [-1, img_size, img_size, img_chan])
X_train = X_train.astype(np.float32) / 255
X_test = np.reshape(X_test, [-1, img_size, img_size, img_chan])
X_test = X_test.astype(np.float32) / 255

Categorical = tf.keras.utils.to_categorical
y_train = Categorical(y_train)
y_test = Categorical(y_test)

ind = np.random.permutation(X_train.shape[0])
X_train, y_train = X_train[ind], y_train[ind]

SPLIT = 0.1
n = int(X_train.shape[0] * (1-SPLIT))
X_valid = X_train[n:]
X_train = X_train[:n]
y_valid = y_train[n:]
y_train = y_train[:n]



def model(x, logits=False, training=False):
    with tf.variable_scope('conv0'):
        z = tf.layers.conv2d(x, filters=32, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('conv1'):
        z = tf.layers.conv2d(z, filters=64, kernel_size=[3, 3],
                             padding='same', activation=tf.nn.relu)
        z = tf.layers.max_pooling2d(z, pool_size=[2, 2], strides=2)

    with tf.variable_scope('flatten'):
        shape = z.get_shape().as_list()
        z = tf.reshape(z, [-1, np.prod(shape[1:])])

    with tf.variable_scope('mlp'):
        z = tf.layers.dense(z, units=128, activation=tf.nn.relu)
        z = tf.layers.dropout(z, rate=0.25, training=training)

    LOGITS = tf.layers.dense(z, units=10, name='logits')
    y = tf.nn.softmax(LOGITS, name='ybar')

    if logits:
        return y, LOGITS
    return y


class Dummy:
    pass


res = Dummy()


with tf.variable_scope('model', reuse=tf.AUTO_REUSE):
    res.x = tf.placeholder(tf.float32, (None, img_size, img_size, img_chan),
                           name='x')
    res.y = tf.placeholder(tf.float32, (None, n_classes), name='y')
    res.training = tf.placeholder_with_default(False, (), name='mode')

    res.ybar, logits = model(res.x, logits=True, training=res.training)

    with tf.variable_scope('acc'):
        count = tf.equal(tf.argmax(res.y, axis=1), tf.argmax(res.ybar, axis=1))
        res.acc = tf.reduce_mean(tf.cast(count, tf.float32), name='acc')

    with tf.variable_scope('loss'):
        xent = tf.nn.softmax_cross_entropy_with_logits(labels=res.y,
                                                       logits=logits)
        res.loss = tf.reduce_mean(xent, name='loss')

    with tf.variable_scope('train_op'):
        optimizer = tf.train.AdamOptimizer()
        vs = tf.global_variables()
        res.train_op = optimizer.minimize(res.loss, var_list=vs)

    res.saver = tf.train.Saver()

    res.x_fixed = tf.placeholder(
        tf.float32, (batch_size, img_size, img_size, img_chan),
        name='x_fixed')
    res.adv_eps = tf.placeholder(tf.float32, (), name='adv_eps')
    res.adv_y = tf.placeholder(tf.int32, (), name='adv_y')

    optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
    res.adv_train_op, res.xadv, res.noise = cw(model, res.x_fixed,
                                               y=res.adv_y, eps=res.adv_eps,
                                               optimizer=optimizer)

print('\nInitializing graph')

res.sess = tf.InteractiveSession()
res.sess.run(tf.global_variables_initializer())
res.sess.run(tf.local_variables_initializer())


def evaluate(res, X_data, y_data, batch_size=128):

    print('\nEvaluating')

    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    loss, acc = 0, 0

    for batch in range(databatch):
        print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
        start = batch * batch_size
        end = min(datasample, start + batch_size)
        cnt = end - start
        batch_loss, batch_acc = res.sess.run(
            [res.loss, res.acc],
            feed_dict={res.x: X_data[start:end],
                       res.y: y_data[start:end]})
        loss += batch_loss * cnt
        acc += batch_acc * cnt
    loss /= datasample
    acc /= datasample

    print(' loss: {0:.4f} acc: {1:.4f}'.format(loss, acc))
    return loss, acc


def train(res, X_data, y_data, X_valid=None, y_valid=None, epochs=1,
          load=False, shuffle=True, batch_size=128, name='model'):

    if load:
        if not hasattr(res, 'saver'):
            return print('\nError: cannot find saver op')
        print('\nLoading saved model')
        return res.saver.restore(res.sess, 'model/{}'.format(name))

    print('\nTrain model')
    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    for epoch in range(epochs):
        print('\nEpoch {0}/{1}'.format(epoch + 1, epochs))

        if shuffle:
            ind = np.arange(datasample)
            np.random.shuffle(ind)
            X_data = X_data[ind]
            y_data = y_data[ind]

        for batch in range(databatch):
            print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
            start = batch * batch_size
            end = min(datasample, start + batch_size)
            res.sess.run(res.train_op, feed_dict={res.x: X_data[start:end],
                                                  res.y: y_data[start:end],
                                                  res.training: True})
        if X_valid is not None:
            evaluate(res, X_valid, y_valid)

    if hasattr(res, 'saver'):
        print('\n Saving model')
        os.makedirs('model', exist_ok=True)
        res.saver.save(res.sess, 'model/{}'.format(name))


def predict(%rehashx, X_data, batch_size=128):

    print('\nPredicting')
    n_classes = res.ybar.get_shape().as_list()[1]

    datasample = X_data.shape[0]
    databatch = int((datasample+batch_size-1) / batch_size)
    yval = np.empty((datasample, n_classes))

    for batch in range(databatch):
        print(' batch {0}/{1}'.format(batch + 1, databatch), end='\r')
        start = batch * batch_size
        end = min(datasample, start + batch_size)
        y_batch = res.sess.run(res.ybar, feed_dict={res.x: X_data[start:end]})
        yval[start:end] = y_batch
    print()
    return yval


def make_cw(res, X_data, epochs=1, eps=0.1, batch_size=batch_size):

    print('\nMaking adversarials via CW')

    datasample = X_data.shape[0]
    databatch = int((datasample + batch_size - 1) / batch_size)
    X_adv = np.empty_like(X_data)

    for batch in range(databatch):
        with Timer('Batch {0}/{1}   '.format(batch + 1, databatch)):
            end = min(datasample, (batch+1) * batch_size)
            start = end - batch_size
            feed_dict = {
                res.x_fixed: X_data[start:end],
                res.adv_eps: eps,
                res.adv_y: np.random.choice(n_classes)}
            res.sess.run(res.noise.initializer)
            for epoch in range(epochs):
                res.sess.run(res.adv_train_op, feed_dict=feed_dict)

            xadv = res.sess.run(res.xadv, feed_dict=feed_dict)
            X_adv[start:end] = xadv

    return X_adv


print('\nTraining')

train(res, X_train, y_train, X_valid, y_valid, load=False, epochs=5,
      name='mnist')

print('\nEvaluating on clean data')

evaluate(res, X_test, y_test)

print('\nGenerating adversarial data')

datasample = 128
ind = np.random.choice(X_test.shape[0], size=datasample, replace=False)
X_test = X_test[ind]
y_test = y_test[ind]

X_adv = make_cw(res, X_test, eps=0.002, epochs=100)

print('\nEvaluating on adversarial data')

evaluate(res, X_adv, y_test)


y1 = predict(res, X_test)
y2 = predict(res, X_adv)

z0 = np.argmax(y_test, axis=1)
z1 = np.argmax(y1, axis=1)
z2 = np.argmax(y2, axis=1)

ind = np.logical_and(z0 == z1, z1 != z2)

ind = z0 == z1

X_test = X_test[ind]
X_adv = X_adv[ind]
z1 = z1[ind]
z2 = z2[ind]
y2 = y2[ind]

ind, = np.where(z1 != z2)
cur = np.random.choice(ind, size=n_classes)
X_org = np.squeeze(X_test[cur])
X_tmp = np.squeeze(X_adv[cur])
y_tmp = y2[cur]

fig = plt.figure(figsize=(n_classes+0.2, 3.2))
gs = gridspec.GridSpec(3, n_classes+1, width_ratios=[1]*n_classes + [0.1],
                       wspace=0.01, hspace=0.01)

label = np.argmax(y_tmp, axis=1)
proba = np.max(y_tmp, axis=1)

for i in range(n_classes):
    ans = fig.add_subplot(gs[0, i])
    ans.imshow(X_org[i], cmap='gray', interpolation='none')
    ans.set_xticks([])
    ans.set_yticks([])

    ans = fig.add_subplot(gs[1, i])
    img = ans.imshow(X_tmp[i]-X_org[i], cmap='RdBu_r', vmin=-1,
                    vmax=1, interpolation='none')
    ans.set_xticks([])
    ans.set_yticks([])

    ans = fig.add_subplot(gs[2, i])
    ans.imshow(X_tmp[i], cmap='gray', interpolation='none')
    ans.set_xticks([])
    ans.set_yticks([])

    ans.set_xlabel('{0} ({1:.2f})'.format(label[i], proba[i]), fontsize=12)

ans = fig.add_subplot(gs[1, n_classes])
dummy = plt.cm.ScalarMappable(cmap='RdBu_r', norm=plt.Normalize(vmin=-1,                                                                vmax=1))
dummy.set_array([])
fig.colorbar(mappable=dummy, cax=ans, ticks=[-1, 0, 1], ticklocation='right')

print('\nSaving figure')

gs.tight_layout(fig)
os.makedirs('img', exist_ok=True)
plt.savefig('img/cw2_mnist.png')

Instructions for updating:
non-resource variables are not supported in the long term

Loading MNIST
11501568/11490434 [==============================] - 0s 0us/step
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/pooling.py:600: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:87: UserWarning: `tf.layers.conv2d` is deprecated and will b


Initializing graph

Training

Train model

Epoch 1/5

Evaluating
 loss: 0.0645 acc: 0.9805

Epoch 2/5
 batch 422/422
Evaluating
 loss: 0.0427 acc: 0.9867

Epoch 3/5
 batch 422/422
Evaluating
 loss: 0.0445 acc: 0.9862

Epoch 4/5

Evaluating
 loss: 0.0424 acc: 0.9858

Epoch 5/5
 batch 422/422
Evaluating
 loss: 0.0360 acc: 0.9890

 Saving model

Evaluating on clean data

Evaluating
 loss: 0.0267 acc: 0.9909

Generating adversarial data

Making adversarials via CW
Batch 1/4   
------- elapsed 3.9777s --------
Batch 2/4   
------- elapsed 3.8179s --------
Batch 3/4   
------- elapsed 3.9130s --------
Batch 4/4   
------- elapsed 3.8011s --------

Evaluating on adversarial data

Evaluating
 loss: 0.8876 acc: 0.5938

Predicting
 batch 1/1

Predicting
 batch 1/1

Saving figure
